In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from keras.layers import Dense, Dropout, Embedding, Activation, Reshape, RepeatVector, LSTM, Bidirectional
from keras.models import Model, Sequential, load_model, save_model
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import text_to_word_sequence
from keras.preprocessing.text import Tokenizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from keras.preprocessing.sequence import pad_sequences
from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder
from sklearn.naive_bayes import GaussianNB 

Using TensorFlow backend.


In [2]:
train = pd.read_csv('../input/train.csv', header=0, sep=',', quotechar='"', usecols=['target', 'comment_text'])

In [3]:
train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv',)

comments = train

In [4]:
stop_words = set(stopwords.words('english')) 
new_stop_words=set(stop_words)

# adding woudlnt type of words into stopwords list
for s in stop_words:
	new_stop_words.add(s.replace('\'',''))
	pass
	
stop_words=new_stop_words
print("Excluding stopwords ...")

# removing @ from default base filter, to remove that whole word, which might be considered as user or page name
base_filters='\n\t!"#$%&()*+,-./:;<=>?[\]^_`{|}~ '

# comments = pd.concat([train['comment_text'], train['target']], axis=1)

Excluding stopwords ...


In [5]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
def removeStopWords(comment):
   return ' '.join([word.lower() for word in comment.split() if word not in stop_words])
#   row['comment_text'] = ' '.join([lemmatizer.lemmatize(word) for word in row['comment_text'].split()])
#   print(index)

In [6]:
punct_mapping = {"_":" ", "`":" "}
punct = "/-'?!.,#$%\'()*+-/:;<=>@[\\]^_`{|}~" + '""“”’' + '∞θ÷α•à−β∅³π‘₹´°£€\×™√²—–&'
def clean_special_chars(text, punct, mapping):
    for p in mapping:
        text = text.replace(p, mapping[p])    
    for p in punct:
        text = text.replace(p, ' ')     
    return text
# test['comment_text'] = test['comment_text']
comments['comment_text'] = comments['comment_text'].apply(lambda x: clean_special_chars(x, punct, punct_mapping))
comments['comment_text'] = comments['comment_text'].apply(removeStopWords)
# test['comment_text'] = test['comment_text'].apply(lambda x: clean_special_chars(x, punct, punct_mapping))

In [7]:
test['comment_text'] = test['comment_text'].apply(lambda x: clean_special_chars(x, punct, punct_mapping))
test['comment_text'] = test['comment_text'].apply(removeStopWords)
# test.to_csv('test_processed.csv')

In [8]:
def replaceNanNull(text):
  if text==np.nan:
    text=''
  return text
# print(replaceNanNull(np.nan))
comments['comment_text'] = comments['comment_text'].replace(np.nan, '')
test['comment_text'] = test['comment_text'].replace(np.nan, '')
comments['comment_text'].dropna(inplace=True)
test['comment_text'].dropna(inplace=True)

In [9]:
train.info()
# comments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1804874 entries, 0 to 1804873
Data columns (total 45 columns):
id                                     int64
target                                 float64
comment_text                           object
severe_toxicity                        float64
obscene                                float64
identity_attack                        float64
insult                                 float64
threat                                 float64
asian                                  float64
atheist                                float64
bisexual                               float64
black                                  float64
buddhist                               float64
christian                              float64
female                                 float64
heterosexual                           float64
hindu                                  float64
homosexual_gay_or_lesbian              float64
intellectual_or_learning_disability    float

In [10]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Error loading stopwords: <urlopen error [Errno -3]
[nltk_data]     Temporary failure in name resolution>
[nltk_data] Error loading wordnet: <urlopen error [Errno -3] Temporary
[nltk_data]     failure in name resolution>


False

In [11]:
tokenizer = nltk.TreebankWordTokenizer()
lemmatizer = nltk.WordNetLemmatizer()

def lemmatize(text):
  return ' '.join([lemmatizer.lemmatize(word) for word in text.split()])

comments['comment_text'] = comments['comment_text'].apply(lemmatize)
test['comment_text'] = test['comment_text'].apply(lemmatize)

In [12]:
def convertToClassification(target):
  if target>=0.5:
    target = 1
  else:
    target = 0
  return target
comments['target'] = comments['target'].apply(convertToClassification)

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer
max_df = 0.3
min_df = 0.003
texts = comments[comments['target']>=0.2]['comment_text']
tfidf = TfidfVectorizer(min_df = np.int(min_df*texts.shape[0]), max_df = max_df, ngram_range = (1,3))
features = tfidf.fit_transform(texts)
vectorizer = pd.DataFrame(features.todense(), columns=tfidf.get_feature_names())



In [14]:
X = tfidf.transform(comments['comment_text'])
Y = comments['target']
x_train = X
y_train = Y

In [15]:
from sklearn import metrics
from matplotlib import pyplot
from sklearn.metrics import average_precision_score, roc_curve, roc_auc_score, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB, MultinomialNB
logisticRegr = LogisticRegression(C=2)
logisticRegr.fit(x_train, y_train)
# predictions1 = logisticRegr.predict_proba(x_test)
# predictions1 = pd.DataFrame(predictions1)

classifier = MultinomialNB()
classifier.fit(x_train, y_train)
# predictions = classifier.predict_proba(x_test)
# predictions = pd.DataFrame(predictions)

# 

/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [16]:
# from sklearn import svm
# classifier = svm.LinearSVC(dual=False, C = 0.5)
# classifier.fit(x_train, y_train)
# predictions = classifier.predict(x_test)
# # predictions = pd.DataFrame(predictions)

# fpr, tpr, thresholds = roc_curve(y_test, predictions)
# auc = roc_auc_score(y_test, predictions)
# print('AUC: %.3f' % auc)
# # plot no skill
# pyplot.plot([0, 1], [0, 1], linestyle='--')
# # plot the roc curve for the model
# pyplot.plot(fpr, tpr, marker='.')
# # show the plot
# pyplot.show()
# classification_report(y_test, predictions>0.5)

In [17]:
# test.drop(['Unnamed: 0'], axis=1)
# test['comment_text'] = test['comment_text'].replace(np.nan, '')
# test['comment_text'].dropna(inplace=True)
# test['comment_text'] = subData['comment_text'].apply(lemmatize)

In [18]:
final = tfidf.transform(test['comment_text'])
logProba = logisticRegr.predict_proba(final)
NBProba = classifier.predict_proba(final)

In [19]:
logProba = pd.DataFrame(logProba)
NBProba = pd.DataFrame(NBProba)
logProba.rename(index=str, columns={'1':'prediction'})
del NBProba[0]
del logProba[0]
NBProba.rename(index=str, columns={'1':'prediction'})
print(logProba)
logProba = logProba.set_index([test['id']])
NBProba = NBProba.set_index([test['id']])

              1
0      0.043898
1      0.014190
2      0.030241
3      0.013952
4      0.977324
5      0.014774
6      0.020411
7      0.022490
8      0.008589
9      0.020175
10     0.032462
11     0.045941
12     0.033306
13     0.012297
14     0.009216
15     0.006408
16     0.144596
17     0.020058
18     0.165747
19     0.064921
20     0.021583
21     0.017588
22     0.011791
23     0.298244
24     0.815754
25     0.018712
26     0.013827
27     0.010451
28     0.026662
29     0.033588
...         ...
97290  0.019368
97291  0.752601
97292  0.076200
97293  0.005639
97294  0.023220
97295  0.016306
97296  0.014030
97297  0.011967
97298  0.013599
97299  0.023433
97300  0.011985
97301  0.014870
97302  0.014677
97303  0.026359
97304  0.021583
97305  0.070265
97306  0.019214
97307  0.009683
97308  0.023641
97309  0.139521
97310  0.061409
97311  0.077650
97312  0.082159
97313  0.201343
97314  0.022589
97315  0.022022
97316  0.012917
97317  0.030346
97318  0.179470
97319  0.013970

[97320 

In [20]:
ls

__notebook__.ipynb  __output__.json


In [21]:
NBProba['prediction'] = NBProba[1]
logProba['prediction'] = logProba[1]
NBProba = NBProba.drop(1, axis=1)
logProba = logProba.drop(1, axis=1)
logProba = logProba.set_index([test['id']])
NBProba = NBProba.set_index([test['id']])
# NBProba.to_csv('nbproba.csv')
# logProba.to_csv('logProba.csv')
# os.chdir('..')
NBProba.to_csv('submission.csv')
# logProba.to_csv('submission.csv')

In [22]:
# os.listdir()

In [23]:
# sample = pd.read_csv('../input/sample_submission.csv')
# sample.head()